In [ ]:
import os
import sys
import datetime
import pprint

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

In [ ]:
from models.llm import LLM

model = LLM('gpt-4o-mini')
llm = model.load_model()
llm

In [3]:
os.environ["OTEL_SDK_DISABLED"] = "true"
os.environ["CREWAI_TELEMETRY_OPT_OUT"] = "true"

In [ ]:
from crewai import Agent, Task, Crew
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain_community.document_loaders import ArxivLoader
from langchain_core.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from crewai_tools import tool
import arxiv

@tool('arxivSearch')
def search_tool(query: str):
    """Search the arxiv for most recent research papers on a given query"""
    return ArxivAPIWrapper(
        top_k_results=5,
        load_max_docs=10,
        kwargs={"sort_by": arxiv.SortCriterion.SubmittedDate}
    ).run(query)

@tool('arxivDocLoader')
def doc_loader_tool(query: str):
    """Search the arxiv for most recent research papers on a given query"""
    return ArxivLoader(
        query=query,
        load_max_docs=3,
        kwargs={"sort_by": arxiv.SortCriterion.SubmittedDate}
    ).load_and_split()

@tool
def short_summary(input_: str) -> str:
    """This tool generates a concise summary of the provided text, distilling key information and insights. 
    It is useful for quickly understanding the main points of a document or text without reading the entire content.
    
    input_: str - file_path or text to be summarized."""
    document_prompt = PromptTemplate(
        input_variables=["page_content"]+[key for key in input_[0].metadata.keys()],
        template="\n".join([str(key)+": " "{"+key+"}" for key in input_[0].metadata.keys()])+"\n{page_content}"
    )
    document_variable_name = "text"

    TEMPLATE = """
        {user_message}
        "------------\n"
        {text}
        "------------\n"
        {citation}
        Additional Instructions: 
        {additional_instructions}
        Output:
    """
    user_message = "Write a concise summary of the following TEXT:"
    additional_instructions = ""
    citation = """For each part of your answer, indicate which sources most support it via citation keys at the end of sentences, 
        ensure that the citations and references are in Harvard format, using the information provided for each document.
        Add References at the end of the summary."""
    prompt = PromptTemplate.from_template(
            template=TEMPLATE,
            partial_variables={
                "user_message": user_message,
                "additional_instructions": additional_instructions,
                "citation":citation
                }
        )

    chain = load_summarize_chain(
                    llm, 
                    chain_type="stuff", 
                    prompt=prompt,
                    document_prompt=document_prompt,
                    document_variable_name=document_variable_name
                )

research_agent = Agent(
  role='Researcher',
  goal='Search the research papers',
  backstory="""You're a smart literature review assistant at a large research organization.
  You're responsible for finding research papers.""",
  llm=llm,
  allow_delegation=False,
  verbose=True
)

planner_agent = Agent(
  role='Planner',
  goal='Plan the document drafting process by creating a structured outline of the document.',
  backstory="""You're a Planner, skilled in organizing and structuring both simple and complex drafting processes.""",
  llm=llm,
  allow_delegation=False,
  verbose=True
)

writer_agent = Agent(
  role='Writer',
  goal='Given a set of raw information and section outline, draft a coherent and well-structured document.',
  backstory="""You're a writer at a large research organisation.
  You are expert at writing detailed research survey.
  """,
  llm=llm,
  allow_delegation=False,
  tools=[short_summary],
  verbose=True
)

editor_agent = Agent(
  role='Writer',
  goal='Given a set of sections and document outline, draft a coherent and well-structured document.',
  backstory="""You are an experienced editor, expert at literature review submission.""",
  llm=llm,
  allow_delegation=False,
  tools=[short_summary],
  verbose=True
)

research = Task(
  description='For a given user query {user_query}, generate 3 relevant search query. Find the latest research works for all the queries.',
  expected_output='A list of search results as retrieved from the search tools.',
  agent=research_agent,
  tools=[search_tool],
  # human_input=True,
)

plan = Task(
  description='Plan the outline for a document for a given user query {user_query}. If schema is provided, use it to create the outline otherwise do it on your own.',
  expected_output='An outline for the document',
  agent=planner_agent,
  # human_input=True,
)

write_article = Task(
    description="""Write an article about user query {user_query} following the document outline and the retrieved documents.
    Given the document outline and retrieved documents generate the best sections possible. \
    First write all the sections other than Introduction and Conclusion. \
    Add necessary references with proper citations.""",
    expected_output="All the sections of the article.",
    agent=writer_agent,
    context=[plan, research], # Will wait for the output of the two tasks to be completed
    # tools=[short_summary]
)

edit_article = Task(
    description="""Given the document outline and generated sections, generate the best literature review possible. \
    Use content of all the sections to write detailed Introduction and Conclusion sections. \
    Add necessary references with proper citations.""",
    expected_output="A well-drafted article.",
    agent=writer_agent,
    context=[write_article, plan], # Will wait for the output of the two tasks to be completed
    # tools=[short_summary]
)


crew = Crew(
    agents=[research_agent, planner_agent, writer_agent],
    tasks=[research, plan, write_article, edit_article],
    verbose=True,
    # iterations=5
)

result = crew.kickoff(inputs=
                      {"user_query": "Long form text or article generation"})
print(result)